<a href="https://colab.research.google.com/github/b21727283/Koc-Holding-Deep-Learning-Bootcamp/blob/main/Urban_Sounds_Classification(Preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Urban Sounds Classification 
## Preprocessing Step 


In [ ]:
# Importing data from Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Including UrbanSound8k using gzip
import tarfile
my_tar = tarfile.open('/content/gdrive/My Drive/UrbanSounds8k/UrbanSound8K.tar.gz')
my_tar.extractall('.') 
my_tar.close()

In [ ]:
# Adding librosa package for music and audio analysis
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
import numpy as np
import librosa.display

In [ ]:
# Creating create_spectrogram function for audio files
def create_spectrogram(y):
  spec = librosa.feature.melspectrogram(y=y)
  spec_conv = librosa.amplitude_to_db(spec, ref=np.max)
  return spec_conv


In [ ]:
import os

path = "/content/UrbanSound8K/audio"

In [ ]:
# When the audio file that is read is sent, it will send the prepared spectrogram back and save it in their classes.
spectograms = []
classIDs = []

In [ ]:
for fold in os.listdir(path):
  if(fold[:4] == "fold"):
    for audio_file in os.listdir(path + "/" + fold):
      if audio_file.endswith(".wav"):
        classID = audio_file.split('-', 2)
        classIDs.append(classID)
        y, sr = librosa.load(path + "/" + fold + "/" + audio_file)
        spectograms.append(create_spectrogram(y))

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1323
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1103
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1523
  n_fft, y.shape[-1]


In [ ]:
# Plotting audio file by using x(time) axis and y(linear) axis and saving the plots as .png

import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=1)
plt.axis('off')

index = 0;
for fold in os.listdir(path):
  if(fold[:4] == "fold"):
    for audio_file in os.listdir(path + "/" + fold):
      if audio_file.endswith(".wav"):
        file_name = audio_file.split('.', 1)[0]
        img_path = './Images/' + file_name

        librosa.display.specshow(spectograms[index], y_axis='linear', x_axis='time',
                               sr=sr)
        index += 1

        plt.axis('off')

        fig.savefig(img_path, bbox_inches='tight', pad_inches = 0)
        plt.clf()

index = 0;

<Figure size 432x288 with 0 Axes>

In [ ]:
# Serializing objects' structure
import pickle

with open("spectograms", "wb") as fp:
  pickle.dump(spectograms, fp)

with open("classIDs", "wb") as fp:
  pickle.dump(classIDs, fp)

In [ ]:
# Unzipping images.zip

!zip -r images.zip ./Images

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  adding: Images/108357-9-0-44.png (deflated 2%)
  adding: Images/165039-7-5-1.png (deflated 1%)
  adding: Images/171243-9-0-49.png (deflated 1%)
  adding: Images/157866-8-0-13.png (deflated 2%)
  adding: Images/13230-0-0-3.png (deflated 1%)
  adding: Images/146690-0-0-140.png (deflated 7%)
  adding: Images/187920-7-0-0.png (deflated 1%)
  adding: Images/195969-0-0-9.png (deflated 1%)
  adding: Images/177729-0-0-68.png (deflated 1%)
  adding: Images/99157-9-0-31.png (deflated 2%)
  adding: Images/146186-5-0-7.png (deflated 2%)
  adding: Images/128030-3-0-0.png (deflated 6%)
  adding: Images/195451-5-0-9.png (deflated 1%)
  adding: Images/7061-6-0-0.png (deflated 1%)
  adding: Images/200460-6-0-0.png (deflated 6%)
  adding: Images/111048-6-0-0.png (deflated 9%)
  adding: Images/172593-2-0-10.png (deflated 1%)
  adding: Images/166942-0-0-10.png (deflated 0%)
  adding: Images/7389-1-4-11.png (deflated 1%)
  adding: Images/7064-6-0-0.png (defl

In [ ]:
!unzip -q images.zip

In [ ]:
import os
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

In [ ]:
dir_path = "Images"
size = 128 # 128 x 128 boyutuna resizing işlemi

images = []
counter = 0

for image in tqdm(os.listdir(dir_path)): 
  try:
    label = int(image.split('-', 2)[1]) # class id (0-9 arası sayı)

    img_path = os.path.join(dir_path, image)
    img = cv2.imread(img_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_img = cv2.resize(gray_img, (size, size))

    images.append([gray_img, label])
    counter += 1
  except:
    print(image) # error

counter = 0

100%|██████████| 8732/8732 [00:14<00:00, 605.33it/s]


In [ ]:
random.shuffle(images) # Mixing the images

In [ ]:
# Separating images and classes into different lists

X = [] # images
Y = [] # classIDs

for x,y in images:
  X.append(x)
  Y.append(y)

del images # Since we already have the data inside this file, we delete it to create empty space in RAM

In [ ]:
# Each image's size is in size*size*1 type. Since 1- is grayscale image, it calculates how many images there are when we write -1
X = np.array(X).reshape(-1,size,size,1) 
Y = np.array(Y).reshape(-1,1)

In [ ]:
X = X / 255 # Converting pixels to 0-1 range (normalization)

In [ ]:
X.shape

(8732, 128, 128, 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# First, create X_train, y_train and X_temporary and y_temporary datasets from X and y
X_train, X_temporary, y_train, y_temporary = train_test_split(X, Y, train_size=0.8)

# Using the X_temporary and y_temporary dataset we just created create validation and test datasets
X_val, X_test, y_val, y_test = train_test_split(X_temporary, y_temporary, train_size=0.5)

In [ ]:
print(X_val.shape)
print(X_test.shape)
print(X_train.shape)

(873, 128, 128, 1)
(874, 128, 128, 1)
(6985, 128, 128, 1)
